# Calculate Diffusion Coefficient

The Free Energy Profile F(q), for radsorbate hopping along the reaction coordinate can be calculated by the mean energy of insertion of the (spherical) adsorbate molecule using the FF parameters in the planes orthogonal to the reaction coordinate:
$$ F(q) = -k_BT ln\langle e^{-\beta \Delta U} \rangle_q $$
Here the brackets denote averaging the Boltzmann factor ocer the square grids of resolution dx Å resolution.

The 1-dim cage-to-cage hopping rate is given by: 
$$ k_{C_1 \rightarrow C_2} = \kappa \sqrt{\frac{k_B T}{2 \pi m}} \frac{e^{-\beta F(q^*)}}{\int_{cage} e^{-\beta F(q)} \,dq} $$
where m is the mass of the adsorbate, q is the reaction coordinate, F is the SOMETHING Free Energy as a function of the reaction coordinate, T is the temperature (T = 298 K), and $\kappa$ is the Bennett-Chandler dynamic correction. $\kappa$ = 1 is a good approximatin for infinite dilution. 
The space is partitioned such that the dividing surface is perpendicular to the reactin coordinate and passes through the location of the maximum free energy barrier along the path F(q*). 


To get the self-diffusion coefficient ($D_s$) 
$$ D_s = \frac{\kappa}{2d} \lambda^2 k_{C_1 \rightarrow C_2} $$

$\lambda$ is the cage-center to cage-center lattice distance.

In [1]:
using PorousMaterials
using Statistics
using PyPlot
using Cubature
using Printf 

┌ Warning: Error requiring `OffsetArrays` from `ArrayInterface`
│   exception = (UndefVarError(:IdOffsetRange), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x00007fc6b7b8dcef, Ptr{Nothing} @0x00007fc6b7c20d24, Ptr{Nothing} @0x00007fc6a99abda2, Ptr{Nothing} @0x00007fc6b7c03769, Ptr{Nothing} @0x00007fc6b7c1ff15, Ptr{Nothing} @0x00007fc6b7c1fbce, Ptr{Nothing} @0x00007fc6b7c20811, Ptr{Nothing} @0x00007fc6b7c21297, Base.InterpreterIP in top-level CodeInfo for ArrayInterface at statement 11, Ptr{Nothing} @0x00007fc6b7c3cb31, Ptr{Nothing} @0x00007fc6b7c3e949, Ptr{Nothing} @0x00007fc6588d72b1, Ptr{Nothing} @0x00007fc6588d72dc, Ptr{Nothing} @0x00007fc6b7c03769, Ptr{Nothing} @0x00007fc6b7c1ff15, Ptr{Nothing} @0x00007fc6b7c1fbce, Ptr{Nothing} @0x00007fc6b7c20811, Ptr{Nothing} @0x00007fc6b7c20b90, Ptr{Nothing} @0x00007fc6b7c2104a, Base.InterpreterIP in MethodInstance for err(::Any, ::Module, ::String) at statement 2, Ptr{Nothing} @0x00007fc6588d7227, Ptr{Nothing} @0x00007fc6588d723c, Ptr

In [2]:
const R = 8.31446261815324 / 1000 # Ideal Gas Constant, units: kJ/(mol-K)
temp = 298.0 # units: K
β = 1 / (R * temp) # units: (kJ/mol)⁻¹

ljff = LJForceField("UFF") # r_cut = 14.0 Å, mixing_rule = Lorentz-Berthelot


adsorbates = Dict(:Xe => Dict(:molecule => Molecule("Xe"),
                              :mol_mass => 131.293 / 1000), # kg/mol
                  :Kr => Dict(:molecule => Molecule("Kr"), 
                              :mol_mass => 83.798 / 1000))  # kg/mol

xtals = Dict(:nipyc   => Crystal("NiPyC2_experiment.cif"),
             :nipycnh => Crystal("Pn_Ni-PyC-NH2.cif"))

┌ Info: Crystal NiPyC2_experiment.cif has Pn space group. I am converting it to P1 symmetry.
│         To prevent this, pass `convert_to_p1=false` to the `Crystal` constructor.
└ @ Xtals /home/ng/.julia/packages/Xtals/DSCSR/src/crystal.jl:433
┌ Info: Crystal Pn_Ni-PyC-NH2.cif has Pn space group. I am converting it to P1 symmetry.
│         To prevent this, pass `convert_to_p1=false` to the `Crystal` constructor.
└ @ Xtals /home/ng/.julia/packages/Xtals/DSCSR/src/crystal.jl:433


Dict{Symbol, Crystal} with 2 entries:
  :nipyc   => Name: NiPyC2_experiment.cif…
  :nipycnh => Name: Pn_Ni-PyC-NH2.cif…

# Monte Carlo Integration TST Method 
1. Determine integration bounds: 
    We will integrate over the entire unit cell, but only points that are 
    accessible will contribute to the integral. Inaccessible points will be rejected. 
    1. calculate grid = energy_grid()
    2. get seg_grid = PorousMaterials._segment_grid()
    3. get acc_grid = calculate_accessibility_grid
    4. get fractional coordinates of molecule (e.g. xf = rand(3))
    5. check that we are in the channel that we want AND the gridpoint is accessible
        - IF seg_grid.data(xf_to_id(xtal.box, xf) == channel_ID && accessible(acc_grid, xf)
             THEN perform calculation vdw_energy(xf)
          ELSE reject point (return exp() = 0? or just return nothing?)
2. Calculate average Boltzmann Factor of the Free energy 

In [3]:
grid_resolution = 0.1 # units: Å
energy_cutoff = 50.0 # units: kJ/mol

grid = energy_grid(xtals[:nipyc], adsorbates[:Xe][:molecule], ljff, resolution=grid_resolution)

Computing energy grid of Xe in NiPyC2_experiment.cif
	Regular grid (in fractional space) of 64 by 127 by 104 points superimposed over the unit cell.


Regular grid of 64 by 127 by 104 points superimposed over a unit cell and associated data.
	units of data attribute: kJ_mol
	origin: [0.000000, 0.000000, 0.000000]


In [4]:
segmented_grid = PorousMaterials._segment_grid(grid, energy_cutoff, true)

Found 2 segments


Regular grid of 64 by 127 by 104 points superimposed over a unit cell and associated data.
	units of data attribute: Segment_No
	origin: [0.000000, 0.000000, 0.000000]


In [5]:
# so the fact that there is a connection between channels makes things somewhat more complicated
#         Noted seg. 3 --> 1 connection in (1, 0, 0) direction
# the nice thing is that it is in the (1, 0, 0) direction while 
# the other channels are in the (0, 0, 1) direction. 
# The reason this makes it easier is that Ds = (Dx + Dy + Dz) / 3  (see refs.)
# meaning that I can do the calculations for each channel independently, 
# disregarding the channels that are not accessible to each other 
# (this is to say that a molecule in channel 1 cannot diffuse into channel 2)
# note: channel 2 is essentially identical to channel 1, bit it is staggered (see Egrid figs in paper)
# could be useful to compare the diffusion in channel 2 to channel 1 (maybe take the average of the two?)

# Oh, wait, but Noted seg. 3 --> 3 connection in (0, 0, 1) direction.
# so, I'm not sure what this channel is... maybe there is another partial channel 
# present in the unit cell of nipycnh..?

acc_grid, nb_segs_blocked, porosity = compute_accessibility_grid(xtals[:nipyc], 
                                                                 adsorbates[:Xe][:molecule], 
                                                                 ljff;
                                                                 resolution=grid_resolution,
                                                                 energy_tol=energy_cutoff,  
                                                                 energy_units=:kJ_mol,
                                                                 verbose=true, 
                                                                 write_b4_after_grids=false,
                                                                 block_inaccessible_pockets=true)

Computing accessibility grid of NiPyC2_experiment.cif using 50.000000 kJ_mol potential energy tol and Xe probe...
Computing energy grid of Xe in NiPyC2_experiment.cif
	Regular grid (in fractional space) of 64 by 127 by 104 points superimposed over the unit cell.
Found 2 segments
Noted seg. 1 --> 1 connection in (1, 0, 0) direction.
Noted seg. 2 --> 2 connection in (1, 0, 0) direction.
	Found 4 simple cycles in segment connectivity graph.
	...found a cycle of accessible segments!
	...found a cycle of accessible segments!
	...found a cycle of accessible segments!
	...found a cycle of accessible segments!
Segment 1 classified as accessible channel.
Segment 2 classified as accessible channel.


(Regular grid of 64 by 127 by 104 points superimposed over a unit cell and associated data.
	units of data attribute: accessibility
	origin: [0.000000, 0.000000, 0.000000]
, 0, Dict(:b4_blocking => 0.07922991747425802, :after_blocking => 0.07922991747425802))

In [6]:
# get list of connections present in segmented grid
# connections.str       start
# connections.dst       destination
# connections.direction direction
connections = PorousMaterials._build_list_of_connections(segmented_grid)

Noted seg. 1 --> 1 connection in (1, 0, 0) direction.
Noted seg. 2 --> 2 connection in (1, 0, 0) direction.


4-element Vector{PorousMaterials.SegmentConnection}:
 PorousMaterials.SegmentConnection(1, 1, (1, 0, 0))
 PorousMaterials.SegmentConnection(1, 1, (-1, 0, 0))
 PorousMaterials.SegmentConnection(2, 2, (1, 0, 0))
 PorousMaterials.SegmentConnection(2, 2, (-1, 0, 0))

In [7]:
# The integrand is the average free energy on a plane along the reaction coord. q
# this average is obtained by MC integration 
function integrand(q::Float64, xtal::Crystal, adsorbate::Molecule,
                  seg_grid::Grid{Int64}, acc_grid::Grid{Bool}; 
                  seg_id::Int=1, nb_insertions::Int=50)
    # initialize array to store energy values
    sample_energy_on_slice = zeros(nb_insertions)
    
    # make a copy incase we need to reset
    molecule = deepcopy(adsorbate)
    
    # make sure that xtal is replicated enough for vdw calcs
    rep_factors = replication_factors(xtal, ljff)
    crystal = replicate(xtal, rep_factors)
    
    # make sure that molecule is in fractional coordinates
    if isa(molecule, Molecule{Cart})
        molecule = Frac(molecule, crystal.box)
    else
        translate_to!(molecule, Frac(molecule.com.xf ./ rep_factors)) # rescale fractional coords
    end
    
    # perform MC integration
    for n in 1:nb_insertions
        # get random location on the slice
        e2, e3 = rand(2) # fractional coords on slice
        xf = [q, e2, e3]
        xf_id = xf_to_id(seg_grid.n_pts, xf)
        
        # make sure to translate the molecule to the location 
        # inside the original unit cell.
        translate_to!(molecule, Frac(molecule.com.xf ./ rep_factors))
        
        if seg_grid.data[xf_id] == seg_id && accessible(acc_grid, xf)
            # xf::Array{Float64, 1}
            function energy()
                @assert xf
                energy = vdw_energy(crystal, molecule, ljff) * 8.314 / 1000 # units: kJ/mol
                return energy
            end
            
            energy = grid.data[xf_id] # energy(xf)
            push!(sample_energy_on_slice, energy)
        end
    end
    avg_boltzmann_factor_on_slice = mean(sample_energy_on_slice)
    return avg_boltzmann_factor_on_slice
end

integrand (generic function with 1 method)

In [10]:
###
#  perform 1-dim integration along channels
###
# bounds in fractional coordinates
lower_bound = 0.0
upper_bound = 1.0

avg_fe, err = hquadrature(xf -> begin integrand(xf, xtals[:nipyc], adsorbates[:Xe][:molecule], 
                                                segmented_grid, acc_grid,); end, 
                          lower_bound, upper_bound)

(0.0, 0.0)

In [9]:
# """
# # Arguments:
# `x::Float64`: reaction coordinate
# `xtal::Crystal`: the MOF
# `temp::Float64`: temperature, units: K. default=298.0 K
# `bounds::Union{Nothing, Vector{Vector{Float64}}}`: 2d integration bounds
# `energy_tol::Float64`: energy tolerance used for determining integratin bounds if bounds are not passed, 
#                         units: kJ/mol
# """
# function free_energy(x::Float64, xtal::Crystal, 
#                      molecule::Molecule, 
#                      ljff::LJForceField; 
#                      temp::Float64=298.0, 
#                      bounds::Union{Nothing, Vector{Vector{Float64}}}=nothing,
#                      energy_tol::Float64=50.0)

#     β = 1 / (R * temp) # units: (kJ / mol)^-1
    
#     # get integration bounds
#     if isnothing(bounds)
#         bounds = get_2d_integration_bounds(xtal, energy_tol) # [[lb_y, lb_z], [ub_y, ub_z]]
#     end
#     # calculate area to normalize integral
#     area = (bounds[2][1] - bounds[1][1]) * (bounds[2][2] - bounds[1][2])
    
#     # integrate planar slices along the reaction coordinate
#     function integrand(yz::Vector{Float64})
#         xf = [x, yz[1], yz[2]] # position in MOF
        
#         # make sure molecule is in fractional coords
#         if isa(molecule, Molecule{Cart})
#             molecule = Frac(molecule, xtal.box)
#         end
        
#         # make sure the coords are fractional
#         xf = mod.(xf, 1.0)
#         # move probe molecule
#         translate_to!(molecule, Frac(xf))
#         # calculate the guest-host VDW interaction
#         # vdw_energy * 8.314 / 1000 gives units: kJ/mol
#         boltzmann_factor = exp(-β * (vdw_energy(xtal, molecule, ljff) * 8.314 / 1000))
#         return boltzmann_factor
#     end # integrand

#     # perform integration
#     fdim = 3 # perform two (y and z dir) real-valued integrals simultaneously
#     (val, err) = hcubature(yz -> begin integrand(yz); end, bounds[1], bounds[2])
#     # return free energy
#     return (-1.0 / β) * log(val / area), err
# end # free_energy